In [2]:
## importing the relevant packages:

# clear the workspace
#%reset -f

# print list of files in directory
import os
print(os.listdir())

# the base packages
import collections # for the Counter function
import csv # for reading/writing csv files
import pandas as pd, numpy as np, time, gc, bisect, re
from datetime import datetime as dt

# the various packages/modules used across processing (sklearn), modelling (lightgbm) and bayesian optimization (hyperopt, bayes_opt)
import sklearn
from sklearn import metrics, preprocessing
import sklearn.decomposition as decomposition
from sklearn.model_selection import cross_val_score
from sklearn.base import TransformerMixin
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
import category_encoders as ce
from scipy.stats import truncnorm

# the modelling packages and related
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

# hyperopt modules
#from bayes_opt import BayesianOptimization
from tqdm import tqdm
from hyperopt import hp, tpe, STATUS_OK, Trials, space_eval, rand
from hyperopt.fmin import fmin
from hyperopt.pyll.stochastic import sample

MAX_EVALS = 5
randomseed = 5 # the value for the random state used at various points in the pipeline
pd.options.display.max_rows = 1000 # specify if you want the full output in cells rather the truncated list
pd.options.display.max_columns = 200

# to display multiple outputs in a cell without usin print/display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

['conf', 'ganglia', 'derby.log', 'logs', 'eventlogs']

In [3]:
## HELPER FUNCTIONS CLASS ##

class helper_funcs():
    
    def __init__():
        """ helper functions used across the pipeline """
    
    ## datetime feature engineering
    def datetime_feats(train, valid):
        cols = [s for s in train.columns.values if 'date' in s]
        print('datetime feature engineering is happening ...', '\n')
        # nested function to derive the various datetime features for a given date column
        def dt_feats(df, col):
            df[col] = pd.to_datetime(df[i])
            #df[str(col+'_'+'day')] = df[col].dt.day
            #df[str(col+'_'+'day_name')] = df[col].dt.day_name
            #df[str(col+'_'+'dayofweek')] = df[col].dt.dayofweek
            df[str(col+'_'+'dayofyear')] = df[col].dt.dayofyear
            #df[str(col+'_'+'days_in_month')] = df[col].dt.days_in_month
            df[str(col+'_'+'month')] = df[col].dt.month
            #df[str(col+'_'+'month_name')] = df[col].dt.month_name
            df[str(col+'_'+'quarter')] = df[col].dt.quarter
            #df[str(col+'_'+'week')] = df[col].dt.week
            #df[str(col+'_'+'weekday')] = df[col].dt.weekday
            df[str(col+'_'+'year')] = df[col].dt.year
            #df[col] = df[col].dt.date
            df = df.drop([col], axis = 1)
            return df
        # loop function over all raw date columns
        for i in cols:
            train = dt_feats(train, i)
            valid = dt_feats(valid, i)
        return train, valid
    
    ## function to get frequency count of elements in a vector/list
    def freq_count(input_vector):
        return collections.Counter(input_vector)
    
    # removing near zero variance columns
    def variance_threshold_selector(train, valid, threshold):
        print('input data shape is: ', train.shape, '\n')
        selector = VarianceThreshold(threshold)
        selector.fit(np.asanyarray(train))
        X = train[train.columns[selector.get_support(indices=True)]]
        Y = valid[valid.columns[selector.get_support(indices=True)]]
        #display(pd.DataFrame(X.head(5)))
        print('output data shape is: ', X.shape, '\n')
        return X, Y

In [4]:
## MISSING VALUE IMPUTATION CLASS ##

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.
        Columns of dtype object are imputed with the most frequent value 
        in column.
        Columns of other types are imputed with mean of column.
        """
        
    def fit(self, X, y=None):
        X_temp = X.copy()
        #self.fill = pd.Series([X_temp.groupby(['pay_grade_group_pa', 'global_job_om'])[c].value_counts().index[0] if X_temp[c].dtype == np.dtype('O') else X_temp.groupby(['pay_grade_group_pa', 'global_job_om'])[c].mean() for c in X_temp], index=X_temp.columns)
        #X_temp = self.fill.copy().reset_index()
        #self.fill = pd.Series([X_temp.groupby(['pay_grade_group_pa'])[c].value_counts().index[0] if X_temp[c].dtype == np.dtype('O') else X_temp.groupby(['pay_grade_group_pa'])[c].mean() for c in X_temp], index=X_temp.columns)
        #X_temp = self.fill.copy().reset_index()
        self.fill = pd.Series([X_temp[c].value_counts().index[0] if X_temp[c].dtype == np.dtype('O') else X_temp[c].mean() for c in X_temp], 
                              index=X_temp.columns)
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
    def num_missing(self):
        return sum(self.isnull())

In [23]:
# function to bucket sparse levels in categorical features to the 'others' category as well as handle new values in the valid df

from sklearn.base import TransformerMixin, BaseEstimator
from collections import defaultdict

class CategoryGrouper(BaseEstimator, TransformerMixin):  
    """A tranformer for combining low count observations for categorical features.
    This transformer will preserve category values that are above a certain threshold, while bucketing together all the other values. This will fix issues where new data may have an unobserved category value that the training data did not have.
    """
    
    def __init__(self, threshold=0.05):
        """ Initialize method.
        Args: threshold (float): The threshold to apply the bucketing when categorical values drop below that threshold.
        """
        self.d = defaultdict(list)
        self.threshold = threshold

    def transform(self, X, **transform_params):
        """Transforms X with new buckets.
        Args: X (obj): The dataset to pass to the transformer.
        Returns: The transformed X with grouped buckets.
        """
        X_copy = X.copy()
        for col in X_copy.columns:
            X_copy[col] = X_copy[col].apply(lambda x: x if x in self.d[col] else 'others')
        return X_copy

    def fit(self, X, y=None, **fit_params):
        """ Fits transformer over X.
        Builds a dictionary of lists where the lists are category values of the
        column key for preserving, since they meet the threshold.
        """
        df_rows = len(X.index)
        for col in X.columns:
            calc_col = X.groupby(col)[col].agg(lambda x: (len(x) * 1.0) / df_rows)
            self.d[col] = calc_col[calc_col >= self.threshold].index.tolist()
        return self

In [24]:
# dfs with 100 elements in cat1 and cat2
# note how df_test has elements 'g' and 't' in the respective categories (unknown values)
df_train = pd.DataFrame({'cat1': ['a'] * 20 + ['b'] * 30 + ['c'] * 40 + ['d'] * 3 + ['e'] * 4 + ['f'] * 3,
                         'cat2': ['z'] * 25 + ['y'] * 25 + ['x'] * 25 + ['w'] * 20 +['v'] * 5})
df_test = pd.DataFrame({'cat1': ['a'] * 10 + ['b'] * 20 + ['c'] * 5 + ['d'] * 50 + ['e'] * 10 + ['g'] * 5,
                        'cat2': ['z'] * 25 + ['y'] * 55 + ['x'] * 5 + ['w'] * 5 + ['t'] * 10})

In [25]:
catgrouper = CategoryGrouper()
catgrouper.fit(df_train)
df_test_transformed = catgrouper.transform(df_test)
df_train_transformed = catgrouper.transform(df_train)

df_train_transformed

Out[91]: CategoryGrouper(threshold=0.05)
Out[91]: 
 cat1 cat2
0 a z
1 a z
2 a z
3 a z
4 a z
5 a z
6 a z
7 a z
8 a z
9 a z
10 a z
11 a z
12 a z
13 a z
14 a z
15 a z
16 a z
17 a z
18 a z
19 a z
20 b z
21 b z
22 b z
23 b z
24 b z
25 b y
26 b y
27 b y
28 b y
29 b y
30 b y
31 b y
32 b y
33 b y
34 b y
35 b y
36 b y
37 b y
38 b y
39 b y
40 b y
41 b y
42 b y
43 b y
44 b y
45 b y
46 b y
47 b y
48 b y
49 b y
50 c x
51 c x
52 c x
53 c x
54 c x
55 c x
56 c x
57 c x
58 c x
59 c x
60 c x
61 c x
62 c x
63 c x
64 c x
65 c x
66 c x
67 c x
68 c x
69 c x
70 c x
71 c x
72 c x
73 c x
74 c x
75 c w
76 c w
77 c w
78 c w
79 c w
80 c w
81 c w
82 c w
83 c w
84 c w
85 c w
86 c w
87 c w
88 c w
89 c w
90 CategoryGrouperOther w
91 CategoryGrouperOther w
92 CategoryGrouperOther w
93 CategoryGrouperOther w
94 CategoryGrouperOther w
95 CategoryGrouperOther v
96 CategoryGrouperOther v
97 CategoryGrouperOther v
98 CategoryGrouperOther v
99 CategoryGrouperOther v